# 38) Differential Equations

## Last time

* Computing derivatives 
* Ill-conditioned optimization
* Recap on Finite Differences

## Today

1. Ordinary differential equations (ODE)
2. Explicit methods: Forward Euler method
3. Higher-order time-stepping schemes: Runge-Kutta 4 (RK4)
4. Stability
5. Implicit methods: Backward Euler

In [ ]:
using LinearAlgebra
using Plots
default(linewidth=4, legendfontsize=12)
using LaTeXStrings

struct RKTable
    A::Matrix
    b::Vector
    c::Vector
    function RKTable(A, b)
        s = length(b)
        A = reshape(A, s, s)
        c = vec(sum(A, dims=2))
        new(A, b, c)
    end
end

function rk_stability(z, rk)
    s = length(rk.b)
    1 + z * rk.b' * ((I - z*rk.A) \ ones(s))
end

rk4 = RKTable([0 0 0 0; .5 0 0 0; 0 .5 0 0; 0 0 1 0], [1, 2, 2, 1] / 6)
heun = RKTable([0 0; 1 0], [.5, .5])
Rz_theta(z, theta) = (1 + (1 - theta)*z) / (1 - theta*z)

function ode_rk_explicit(f, u0; tfinal=1, h=0.1, table=rk4)
    u = copy(u0)
    t = 0.
    n, s = length(u), length(table.c)
    fY = zeros(n, s)
    thist = [t]
    uhist = [u0]
    while t < tfinal
        tnext = min(t+h, tfinal)
        h = tnext - t
        for i in 1:s
            ti = t + h * table.c[i]
            Yi = u + h * sum(fY[:,1:i-1] * table.A[i,1:i-1], dims=2)
            fY[:,i] = f(ti, Yi)
        end
        u += h * fY * table.b
        t = tnext
        push!(thist, t)
        push!(uhist, u)
    end
    thist, hcat(uhist...)
end

## 1. Ordinary Differential Equations

Given **initial condition** $y(t=0)=y(0)=y_0$, we want to find $y(t)$ for $t > 0$ that satisfies

$$ y' \equiv \dot y \equiv \frac{\partial y}{\partial t} = f(t, y) $$

| Application | $y$ | $f$ |
| --- | --- | --- |
| Orbital dynamics | position, momentum | conservation of momentum|
| Chemical reactions | concentration | conservation of atoms |
| Epidemiology | infected/recovered population | transmission and recovery |

* $y$ can be a scalar or a vector

## Solving differential equations


### Linear equations

$$ y' = A(t) y + \text{sink}(t)/\text{source}(t)$$

* Autonomous if $A(t) \equiv A$ and source independent of $t$: $ y' = A y + \text{sink}/\text{source}$
* Non-autonomous otherwise

* Suppose $y$ and $a = A$ are _scalars_:

$$ y' = a y$$

then there is an an _analytic solution_:

$$y(t) = e^{at} y_0$$

* We can do the same for systems

$$ y(t) = e^{A t} y_0 $$

### But what does it mean to exponentiate a matrix?

Taylor series again!

$$ e^A = 1 + A + \frac{A^2}{2} + \frac{A^3}{3!} + \dotsb $$

where here $1$ means the identity matrix $I$, and there are many [practical ways to compute it](https://bpb-us-e1.wpmucdn.com/blogs.cornell.edu/dist/c/9924/files/2021/10/19ways.pdf) (a paper co-authored by the founder of MATLAB).

#### Exponentiate a diagonalizable matrix
Suppose that the diagonalization $A = X \Lambda X^{-1}$ exists, how do we derive a finite expression for the matrix exponential of $A$, i.e., $e^A$, using the scalar `exp` function?

**Solution**:

We have:
$$ e^A = 1 + A + \frac{A^2}{2} + \frac{A^3}{3!} + \dotsb $$

$$A^n = (X \Lambda X^{-1})^n = X \Lambda X^{-1} \cdot X \Lambda X^{-1} \cdot X \Lambda X^{-1} \cdot \ldots \cdot X \Lambda X^{-1} = X \Lambda^n X^{-1}$$

Thus,

$$
e^A = X X^{-1} + X \Lambda X^{-1} + X \Lambda^2 X^{-1}/2 + \dots
$$

Factoring out the $X$ and $X^{-1}$ on the left and right gives:

$$e^A = X (I + \Lambda  + \Lambda^2/2 + \dots) X^{-1} = X e^{\Lambda}X^{-1}.$$



## 2. Explicit methods for solving ODEs: Forward Euler method

The simplest method for solving 

$$y'(t) = f(t,y)$$

is to use numerical differentiation to approximate the derivative.

Recall: We know the definition of the difference quotient from Calculus:

$$y' = \lim_{h\to 0} \frac{y(t+h) - y(t)}{h}$$

Hence, we can use a **forward difference** formula:

$$
\frac{\partial {y^n}^F}{\partial t} \approx \frac{ y^{n+1} - y^{n}}{h} + O(h) \equiv \frac{ y^{n+1} - y^{n}}{\Delta t} + O(\Delta t)
$$

this is a _first-order_ approximation, where $h \equiv \Delta t$ is the step size, and we have approximated the time domain (temporal axis) with uniform grid spacing $\Delta t$, so that $y^n = n \Delta t$, for $n = 0, \ldots, N$.

When $n=0$ we use the **initial condition** given: $y(t=0)=y(0)=y_0$

$$ y' \approx \frac{y(h) - y(0)}{h} $$

which yields 

$$ y' \approx \frac{y(h) - y(0)}{h} = f(0, y(0)) $$

Hence, we have the solution estimate

$$ \tilde y(h) = y(0) + h f(0, y(0)) $$





Let's try this on a scalar problem

$$ y' = -k (y - \cos t) $$

where $k$ is a parameter controlling the rate at which the solution $y(t)$ is pulled toward the curve $\cos t$.

In [ ]:
function ode_euler(f, y0; tfinal=10., h=0.1)
    y = copy(y0)
    t = 0.
    thist = [t]
    yhist = [y0]
    while t < tfinal
        tnext = min(t+h, tfinal)
        h = tnext - t
        y += h * f(t, y)
        t = tnext
        push!(thist, t)
        push!(yhist, y)
    end
    thist, hcat(yhist...)
end

In [ ]:
f1(t, y; k=5) = -k * (y .- cos(t))

thist, yhist = ode_euler(f1, [1.], tfinal=10, h=.25)
scatter(thist, yhist[1,:], label = "Forward Euler")
plot!(cos, label = L"\cos(x)")

### Forward Euler on a linear system

If we have a system of ODEs:

$$
\dot{\mathbf{y}} = A \mathbf{y}
$$

For example, a system of two first-order linear ODEs:

\begin{align}
y_1' &= y_2\\
y_2' &= - y_1\\
\end{align}

In matrix form: 


$$ \begin{bmatrix} y_1 \\ y_2 \end{bmatrix}' = \begin{bmatrix} 0 & 1 \\ -1 & 0 \end{bmatrix} \begin{bmatrix} y_1 \\ y_2 \end{bmatrix}$$

In [ ]:
f2(t, y) = [0 1; -1 0] * y

thist, yhist = ode_euler(f2, [0., 1], h=.02, tfinal=10)
scatter(thist, yhist', label = [L"y_1" L"y_2"])
plot!([cos, sin], label = [L"\cos(x)" L"\sin(x)"])

In [ ]:
eigen([0 1; -1 0])

This problem has imaginary eigenvalues (the solution is an oscillator). Forward Euler on this problem is **not** the best choice: it is amplifying the oscillations!

## 3. Higher-order time-stepping schemes: Runge-Kutta 4 (RK4)

In [ ]:
thist, yhist = ode_rk_explicit(f2, [0., 1], h=0.5, tfinal=10)
scatter(thist, yhist', label = [L"y_1" L"y_2"])
plot!([cos, sin], size=(800, 500), label = [L"\cos(x)" L"\sin(x)"])

* Apparently it is possible to integrate this system using large time steps.
* This method evaluates $f(t)$ _four times per step_, so the cost is about equal when the step size $h$ is 4x larger than forward Euler.

But let's run it again with a larger step size
 

In [ ]:
thist, yhist = ode_rk_explicit(f2, [0., 1], h=1, tfinal=10)
scatter(thist, yhist', label = [L"y_1" L"y_2"])
plot!([cos, sin], size=(800, 500), label = [L"\cos(x)" L"\sin(x)"])

## 4. Stability

To analyze stability, we use _Linear Stability Analysis (LSA)_.

### Linear Stability Analysis

Why did Euler diverge (even if slowly) while RK4 solved this problem accurately?

And why do both methods diverge if the step size is too large?

We can understand the convergence of methods by analyzing the test problem

$$ y' = \lambda y $$

for different values of $\lambda$ in the complex plane.

One step of the Euler method with step size $h$ maps

$$ y \to y + h \lambda y = \underbrace{(1 + h \lambda)}_{R(h \lambda)} y .$$

When does this map cause solutions to "blow up" and when is it stable?

In [ ]:
function plot_stability(Rz, method; xlim=(-3, 2), ylim=(-1.5, 1.5))
    x = xlim[1]:.02:xlim[2]
    y = ylim[1]:.02:ylim[2]
    plot(title="Stability: $method", aspect_ratio=:equal, xlim=xlim, ylim=ylim)
    heatmap!(x, y, (x, y) -> abs(Rz(x + 1im*y)), c=:bwr, clims=(0, 2))
    plot!(x->0, color=:black, linewidth=1, label=:none)
    plot!([0, 0], [ylim...], color=:black, linewidth=1, label=:none)
end

### Stability of Forward Euler

In [ ]:
plot_stability(z -> 1 + z, "Forward Euler")

### Stability for RK4

In [ ]:
plot_stability(z -> rk_stability(4z, rk4), "RK4")


In [ ]:
plot_stability(z -> rk_stability(2z, heun), "Heun's method")

## 5. Implicit methods: Backward Euler 

Recall that forward Euler is the step

$$ \tilde y(h) = y(0) + h f(0, y(0)) . $$

This can be evaluated **explicitly**; all the terms on the right hand side are known so the approximation $\tilde y(h)$ is computed merely by evaluating the right hand side.

Let's consider an alternative, **backward Euler** (or "implicit Euler"). 

Recall the **forward difference** formula:

$$
\frac{\partial {y^n}^F}{\partial t} \approx \frac{ y^{n+1} - y^{n}}{h} + O(h) \equiv \frac{ y^{n+1} - y^{n}}{\Delta t} + O(\Delta t)
$$

If instead we use the **backward difference** formula:

$$
\frac{\partial {y^{n}}^B}{\partial t} \approx \frac{ y^{n} - y^{n-1}}{h} + O(h) \equiv \frac{ y^{n} - y^{n-1}}{\Delta t} + O(\Delta t)
$$

When $n=1$ we use the **initial condition** given: $y(t=t_0=0)=y(0)=y_0$

$$ y'(t=t_1) \approx \frac{y(h) - y(0)}{h} $$

which yields 

$$ y'(t=t_1) \approx \frac{y(h) - y(0)}{h} = f(t=t_1, y(t=t_1)) $$

which yields to the solution estimate

$$ \tilde y(h) = y(0) + h f(h, \tilde y(h)) . $$

This is a (generally) _nonlinear_ equation for $\tilde y(h)$.

### Stability of backward Euler

For the test equation $y' = \lambda y$, the backward Euler method is

$$ \tilde y(h) = y(0) + h \lambda \tilde y(h) $$

or

$$ \tilde y(h) = \underbrace{\frac{1}{1 - h \lambda}}_{R(h\lambda)} y(0) . $$

In [ ]:
plot_stability(z -> 1/(1-z), "Backward Euler")

### Computing with implicit methods

$$ \tilde y(h) = y(0) + h f\big(\tilde y(h) \big) $$

* Linear solve for linear problem
* Nonlinear (often Newton) solve for nonlinear problem
* Need Jacobian or finite differencing

In [ ]:
plot_stability(z -> Rz_theta(z, .5), "Midpoint method")